In [2]:
#Lesum inn gögnin
import numpy as np

gogn = np.loadtxt("Gögn/games.csv", delimiter=",", dtype=str, comments=None)
#display(gogn)
gogn.shape

(20059, 16)

In [3]:
#Eyðum gögnum sem eru ekki notuð
dalkarTilAdEyda = [15,14,13,10,8,7,5,3,2,1]
gogn = np.transpose(gogn)
gogn = np.delete(gogn,dalkarTilAdEyda,0)
gogn = np.transpose(gogn)
gogn.shape

(20059, 6)

In [47]:
#Finnum lengd lengstu skákar
import numpy as np
max_lengd = 0
temp_max_lengd = 0
for i in range(len(gogn[:,0])):
    leikir = gogn[i,5].split()
    temp_max_lengd = len(leikir)
    if temp_max_lengd > max_lengd:
        max_lengd = temp_max_lengd
        maxi = i
print(max_lengd)
print(maxi)

#Setjum skákirnar í fylki, leik fyrir leik
leikir = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(1, len(gogn[:,0])):
    temp_leikir = gogn[i,5].split()
    for j in range(len(temp_leikir)):
        leikir[i-1,j] = temp_leikir[j]
print(leikir.shape)

349
11555
(20058, 349)


In [56]:
#Breytum uppsetingu á leikjum úr (d4 d5...) í bitmap og bætum við númeri leiks
import chess
leikir_fen = np.zeros((len(gogn[:,0])-1,max_lengd), dtype=list)
for i in range(len(leikir[:,0])):
    for j in range(len(leikir[0,:])):
        if leikir[i,j] != 0:
            board = chess.Board()
            for k in range(j):
                board.push_san(leikir[i,k])
            leikir_fen[i,j] = board.fen()
        else:
            break

#board = chess.Board()
#board.push_san('d4')
#board.fen

In [8]:
#a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
#np.savetxt("foo.csv", a, delimiter=",")